In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os
import sys
import os.path

In [3]:
import htpar
reload(htpar)

<module 'htpar' from 'htpar/__init__.pyc'>

# utils.py

In [4]:
assert htpar.split_sharded_path("x-@017-y") == ("x-%03d-y", 17)

In [5]:
assert list(htpar.path_shards("x-@002")) == ["x-000", "x-001"]

In [6]:
assert htpar.splitallext("a/b/c/d.e.f") == ("a/b/c/d", "e.f")

In [7]:
assert htpar.base_plus_ext("a/b/c.d.e") == ("a/b/c", "d.e")

In [8]:
assert htpar.dir_plus_file("a/b/c/d.e.f") == ("a/b/c", "d.e.f")

In [9]:
assert htpar.last_dir("a/b/c/d.e.f") == ("c", "d.e.f")

In [10]:
f = htpar.get_keyfun("(.*)::(.*)")
assert f("a::b") == ("a", "b")

In [11]:
sample = dict(png=rand(100, 100, 3), json=[1, 2])
encoded = htpar.autoencode(sample)
assert encoded["json"] == "[1, 2]"
assert encoded["png"][:7] == "\x89PNG\r\n\x1a"
decoded = htpar.autodecode(encoded)
assert decoded["png"].shape == (100, 100, 3)
assert decoded["json"] == [1, 2]

/usr/local/lib/python2.7/dist-packages/imageio/core/util.py:78: UserWarning: Lossy conversion from float64 to uint8, range [0, 1]
  dtype_str, out_type.__name__))


# tarfiles.py

In [12]:
with htpar.TarRecords("_test.tgz") as stream:
    for i in range(3):
        s = "%03d" % i
        stream.write(dict(__key__=s, s=s))

In [13]:
assert os.path.exists("_test.tgz")

In [14]:
contents = os.popen("tar -ztvf _test.tgz").read()
print contents
assert "002.s" in contents

-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 000.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 001.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 002.s



In [15]:
result = list(htpar.tarrecords("_test.tgz"))
result

[{'__key__': '000', 's': '000'},
 {'__key__': '001', 's': '001'},
 {'__key__': '002', 's': '002'}]

In [16]:
assert len(result) == 3
assert result[0]["s"] == "000"
assert result[2]["__key__"] == "002"

In [17]:
result1 = list(htpar.tarshards("_test.tgz"))
assert result == result1

In [18]:
with htpar.TarShards("_test-@0004.tgz") as stream:
    for i in range(100):
        s = "%03d" % i
        stream.write(dict(__key__=s, s=s))

[opened 4 output streams]


In [19]:
contents = os.popen("tar -ztvf _test-0001.tgz").read()
print contents
assert "030.s" in contents

-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 007.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 009.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 013.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 014.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 017.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 018.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 026.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 028.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 030.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 036.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 038.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 039.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 040.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 042.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 044.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 047.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 052.s
-rw-rw-rw- bigdata/bigdata   3 2018-01-17 22:43 058.s
-rw-rw-rw- bigdata/bigdata  

In [20]:
result = list(htpar.tarshards("_test-@0004.tgz"))
result[:5]

[{'__key__': '001', 's': '001'},
 {'__key__': '004', 's': '004'},
 {'__key__': '005', 's': '005'},
 {'__key__': '010', 's': '010'},
 {'__key__': '011', 's': '011'}]

In [21]:
assert len(result) == 100

In [22]:
assert set(x["s"] for x in result) == \
       set("%03d" % i for i in range(100))

In [23]:
!rm -f _test*

# tarfiles.py with encoders

In [24]:
with htpar.TarRecords("_test.tgz") as stream:
    for i in range(3):
        s = "%03d" % i
        stream.write({"__key__": s, "a.json": [s], "b.pickle": {s: s}, "c.mp": (s, s), "d.png": rand(5,4,3)})

In [25]:
assert os.path.exists("_test.tgz")

In [26]:
contents = os.popen("tar -ztvf _test.tgz").read()
print contents
assert "002.c.mp" in contents

-rw-rw-rw- bigdata/bigdata   7 2018-01-17 22:43 000.a.json
-rw-rw-rw- bigdata/bigdata  20 2018-01-17 22:43 000.b.pickle
-rw-rw-rw- bigdata/bigdata   9 2018-01-17 22:43 000.c.mp
-rw-rw-rw- bigdata/bigdata 133 2018-01-17 22:43 000.d.png
-rw-rw-rw- bigdata/bigdata   7 2018-01-17 22:43 001.a.json
-rw-rw-rw- bigdata/bigdata  20 2018-01-17 22:43 001.b.pickle
-rw-rw-rw- bigdata/bigdata   9 2018-01-17 22:43 001.c.mp
-rw-rw-rw- bigdata/bigdata 133 2018-01-17 22:43 001.d.png
-rw-rw-rw- bigdata/bigdata   7 2018-01-17 22:43 002.a.json
-rw-rw-rw- bigdata/bigdata  20 2018-01-17 22:43 002.b.pickle
-rw-rw-rw- bigdata/bigdata   9 2018-01-17 22:43 002.c.mp
-rw-rw-rw- bigdata/bigdata 133 2018-01-17 22:43 002.d.png



In [27]:
result = list(htpar.tarrecords("_test.tgz"))
print result[1]
assert result[0]["a.json"] == ["000"]
assert result[1]["d.png"].shape == (5, 4, 3)
assert result[2]["c.mp"] == ["002", "002"]
assert result[1]["b.pickle"] == {"001": "001"}

{'a.json': ['001'], 'b.pickle': {'001': '001'}, 'c.mp': ['001', '001'], '__key__': '001', 'd.png': array([[[253,  70, 224],
        [ 62, 192, 198],
        [ 87, 112, 239],
        [ 88, 239, 151]],

       [[176,   3, 117],
        [165,   7, 246],
        [215,  14,  27],
        [ 62, 120, 150]],

       [[ 76, 178,  34],
        [202, 177, 104],
        [ 28, 154,  85],
        [ 91,  93,  59]],

       [[148, 238,  82],
        [141, 252, 103],
        [ 91,  67,  97],
        [149,  61,  84]],

       [[ 88,  65,  43],
        [ 42, 230, 124],
        [225,  59, 223],
        [105, 174,   0]]], dtype=uint8)}
